## Install libraries

In [1]:
!pip install cmake 'gym[atari]' scipy
!pip install gym[atari]
!pip install autorom[accept-rom-license]
!pip install gym[atari,accept-rom-license]==0.21.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441027 sha256=5572d323973437f1e9835c3d365c78c19f42f5d3e30c08c6d1bcdce806730c60
  Stored in directory: /root/.cache/pip/wheels/87/67/2e/6147e7912fe37f5408b80d07527dab807c1d25f5c403a9538a
Successfully built AutoROM.accept-rom-license
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 8.0 MB/s 
     |████████████████████████████████| 1.6 MB 34.6 MB/s 
  Created wheel for

## Import the libraries

In [2]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
from time import sleep
import gym
import numpy as np
import random

## Setup the game environment

In [3]:
def get_env(env_name):
  """ This function takes the environment name and return the environment after resetting 
  input: env_name -> string
  return: env -> the environment object
  """
  env = gym.make(env_name)
  env.reset() # reset environment to a new, random state
  return env

## build frames of the game till it's done

In [6]:
def frame_builder(env):
  """  this function take the env and take actions till the game done and return the frames of the game

  Input:  
      env -> environment object
  Output:
      frames -> list of dictionaries as each frame has [{action, frame, reward, state},....]
  """
  env.render()  
  epochs = 0
  penalties, reward = 0, 0
  frames = []
  done = False

  while not done:
    # automatically selects one random action 
      action = env.action_space.sample()
      state, reward, done, info = env.step(action)

      if reward == -10:
          penalties += 1
      
      # Put each rendered frame into dict for animation
      frames.append({
          'frame': env.render(mode='ansi'),
          'state': state,
          'action': action,
          'reward': reward
          }
      )
      epochs += 1
  # print("Timesteps taken: {}".format(epochs))
  # print("Penalties incurred: {}".format(penalties))
  return frames

In [9]:
env_name = 'Taxi-v3'
env = get_env(env_name)
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))
frames = frame_builder(env)
frames[0]

Action Space Discrete(6)
State Space Discrete(500)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



{'action': 5,
 'frame': '+---------+\n|\x1bR\x1b: | : :G|\n| : | : : |\n| : : : : |\n| | : | : |\n|\x1bY\x1b|\x1b \x1b: |B: |\n+---------+\n  (Dropoff)\n',
 'reward': -10,
 'state': 422}

In [10]:
env_name = 'FrozenLake-v1'
env = get_env(env_name)
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))
frames = frame_builder(env)

Action Space Discrete(4)
State Space Discrete(16)

SFFF
FHFH
FFFH
HFFG


{'action': 1,
 'frame': '  (Down)\nSFFF\n\x1bF\x1bHFH\nFFFH\nHFFG\n',
 'reward': 0.0,
 'state': 4}

## Frame visualization function
visualize the updated frames in the game 

In [11]:
def print_frames(frames):
  """" this fucntion go pass over the frames to show us each frame and it's info
    
  Input: 
      the frames

  print: 
      frame, state, action, and reward
  """
  for i, frame in enumerate(frames):
        # clear_output(wait=True)
        #print(frame['frame'].getvalue())
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)


## Implement Q-Learning

### Build the q-table

In [32]:
def q_table_train(env,alpha = 0.1,gamma = 0.6,epsilon = 0.1, decay_over=False, decay_factor=.1):
  """
  This function is for building the  q-table with trained weights and use the decay over 

  Input :
      alpha (float)-> the learning rate -> scaler
      gamma (float) -> the discount factor -> scaler
      epsilon (float) ->the epsilon-greedy action selection -> scaler

      decay_over -> Boolen varible
      decay_factor -> float to manage the speed of decaying

  Output : 
      q-table (list)
  """

  all_epochs = []
  all_penalties = []
  q_table = np.zeros([env.observation_space.n, env.action_space.n])

  for i in range(1, 100001): #100001
      if decay_over and (i %5000==0):
        alpha, gamma, epsilon = alpha*(1-alpha*decay_factor), gamma*(1-gamma*decay_factor), epsilon*(1-epsilon*decay_factor)
         
      state = env.reset()
      epochs, penalties, reward, = 0, 0, 0
      done = False
      # decay over epsod
      

      while not done:
          if random.uniform(0, 1) < epsilon:
              action = env.action_space.sample() # Explore action space
          else:
              action = np.argmax(q_table[state]) # Exploit learned values

          next_state, reward, done, info = env.step(action) 
          
          old_value = q_table[state, action]
          next_max = np.max(q_table[next_state])
          
          new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
          q_table[state, action] = new_value

          if reward == -10:
              penalties += 1

          state = next_state
          epochs += 1
          
      if i % 100 == 0:
          clear_output(wait=True)
          print(f"Episode: {i}")

  # print("Training finished.\n","alpha= ",alpha," gamma= ", gamma," epsilon= ", epsilon)

  return q_table

In [16]:
def model_evaluate(env, q_table):
    """ 
    the function take the env object and the q-table list to find the AVG_timesteps, and the AVG_penalities

    Input: 
        env (object type) 
        q_table (list)

    Output:
        frames (list)-> list of frames
        AVG_timesteps (float)-> the average time steps
        AVG_penalities (float)-> the average penalites
    """
    frames = []
    total_epochs, total_penalties = 0, 0
    episodes = 100
    for _ in range(episodes):
        state = env.reset()
        epochs, penalties, reward = 0, 0, 0
        done = False

        while not done:
            action = np.argmax(q_table[state])
            state, reward, done, info = env.step(action)
            if reward == -10:
                penalties += 1
            frames.append({
                'frame': env.render(mode='ansi'),
                'state': state,
                'action': action,
                'reward': reward
                }
            )
            epochs += 1

        total_penalties += penalties
        total_epochs += epochs
    
    AVG_timesteps = total_epochs / episodes
    AVG_penalities = total_penalties / episodes

    # print(f"Results after {episodes} episodes:")
    # print(f"Average timesteps per episode: {AVG_timesteps}")
    # print(f"Average penalties per episode: {AVG_penalities}")
    
    return frames, AVG_timesteps, AVG_penalities

## Hyperparameter

In [17]:
alpha = 0.6
gamma = 0.9
epsilon = 0.1

## Q-table

In [18]:
env_name = 'Taxi-v3'
env = get_env(env_name)
q_table=q_table_train(env,alpha =alpha,gamma = gamma,epsilon = epsilon)
frames, AVG_timesteps, AVG_penalities= model_evaluate(env, q_table)
print(AVG_timesteps, AVG_penalities)
print(q_table)

Episode: 100000
Training finished.
 alpha=  0.6  gamma=  0.9  epsilon=  0.1
Results after 100 episodes:
Average timesteps per episode: 12.9
Average penalties per episode: 0.0
12.9 0.0
[[ 0.          0.          0.          0.          0.          0.        ]
 [-0.58568212  0.4603532  -0.58568212  0.4603532   1.62261467 -8.5396468 ]
 [ 4.348907    5.94323     4.348907    5.94323     7.7147     -3.05677   ]
 ...
 [ 6.29214704  9.68288002  5.57103107  5.94323    -6.2802558  -4.73742506]
 [-0.58815544  2.74280265  1.48112538  2.9140163  -7.40773006 -7.34212427]
 [14.3        11.87       14.3        17.          5.3         5.3       ]]


In [19]:
env_name = 'FrozenLake-v1'
env = get_env(env_name)
q_table=q_table_train(env,alpha = alpha,gamma = gamma,epsilon = epsilon)
frames, AVG_timesteps, AVG_penalities= model_evaluate(env, q_table)
print(AVG_timesteps, AVG_penalities)
print(q_table)

Episode: 100000
Training finished.
 alpha=  0.6  gamma=  0.9  epsilon=  0.1
Results after 100 episodes:
Average timesteps per episode: 31.85
Average penalties per episode: 0.0
31.85 0.0
[[3.72854495e-02 1.51541695e-02 1.51120833e-02 1.52592078e-02]
 [1.05489304e-02 8.46983543e-03 1.23331238e-02 1.39917034e-02]
 [1.17548584e-02 1.46766974e-02 1.34105016e-02 1.55141080e-02]
 [1.55075932e-02 1.91469105e-04 4.61398695e-03 1.45935627e-02]
 [5.38697159e-02 1.25660159e-02 9.29676256e-03 1.49991233e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.40652054e-03 2.36477694e-03 1.74857819e-02 1.82020408e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.82714811e-02 1.22459825e-02 9.07095916e-03 1.42757727e-01]
 [6.40709044e-02 1.54867399e-01 2.25864212e-02 3.26410192e-02]
 [3.86103559e-02 7.91237682e-03 2.69532024e-01 4.87721112e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e

## Train model function

In [18]:
def train_model(env_name="Taxi-v3", alpha_para = 0.1, gamma_para = 0.6, epsilon_para = 0.1,decay_over=False, decay_factor=.1):
  """ the function work to train the model using the parameters plus gaving an option to apply the decay over episodes with a decay factor

  Input: 
    env_name (String): the game name
    alpha_para (float), gamma_para (float), epsilon_para (float)

    decay_over (boolean) -> to apply the decay technique or not
    decay_factor (float): due to the decay equation we need the decay_factor, the Equation (parameter*(1-parameter*decay_factor) )
    
  Output:
    frames (list): list of frames
    AVG_timesteps (float)-> the average time steps
    AVG_penalities (float)-> the average penalites
    
  """
  env = get_env(env_name)
  # frames= frame_builder(env)   
  q_table=q_table_train(env,alpha = alpha_para,gamma = gamma_para,epsilon = epsilon_para,decay_over=decay_over,decay_factor=decay_factor)
  frames, AVG_timesteps, AVG_penalities = model_evaluate(env, q_table)
    
  return frames, AVG_timesteps, AVG_penalities

## 2) Tune alpha, gamma, and/or epsilon using a decay over episodes

In [33]:
env_name = 'Taxi-v3'
frames, AVG_timesteps, AVG_penalities = train_model(env_name, alpha_para = 0.1, gamma_para = 0.6, epsilon_para = 0.9,decay_over=True,decay_factor=.1)
print(f"Average timesteps per episode: {AVG_timesteps}")
print(f"Average penalties per episode: {AVG_penalities}")

Episode: 100000
Average timesteps per episode: 38.85
Average penalties per episode: 0.0


In [34]:
print_frames(frames)

Streaming output truncated to the last 5000 lines.
|Y| : |B: |
+---------+
  (West)

Timestep: 3501
State: 366
Action: 3
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)

Timestep: 3502
State: 366
Action: 3
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)

Timestep: 3503
State: 366
Action: 3
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)

Timestep: 3504
State: 366
Action: 3
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)

Timestep: 3505
State: 366
Action: 3
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)

Timestep: 3506
State: 366
Action: 3
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)

Timestep: 3507
State: 366
Action: 3
Reward: -1
+---------+
|R: | : :G

In [35]:
env_name = 'FrozenLake-v1'
frames,AVG_timesteps, AVG_penalities = train_model(env_name, alpha_para = 0.1, gamma_para = 0.6, epsilon_para = 0.9,decay_over=True,decay_factor=.1)
print(f"Average timesteps per episode: {AVG_timesteps}")
print(f"Average penalties per episode: {AVG_penalities}")

Episode: 100000
Average timesteps per episode: 8.31
Average penalties per episode: 0.0


In [36]:
print_frames(frames)

Streaming output truncated to the last 5000 lines.
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 332
State: 0
Action: 2
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 333
State: 4
Action: 2
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 334
State: 8
Action: 2
Reward: 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG

Timestep: 335
State: 8
Action: 1
Reward: 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG

Timestep: 336
State: 12
Action: 1
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 337
State: 4
Action: 2
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 338
State: 8
Action: 2
Reward: 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG

Timestep: 339
State: 12
Action: 1
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 340
State: 4
Action: 2
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 341
State: 5
Action: 2
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 342
State: 4
Action: 2
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 343
State: 0
Action: 2
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timeste

## 3) Implement a grid search to discover the best hyperparameters

In [37]:
def grid_search(env_name="Taxi-v3",parameters={'alpha':[0.9],'gamma':[0.9],'epsilon':[.9]},decay_over=False,decay_factor=.1):
  """ 
  This function try to find the best compination of parmteres with respect to the lowest penalty with minimum timesteps

  Input: 
      env_name (string) -> Game name
      parameters (dict) -> Dictionary of lists for each parameter; Example:{'alpha':[0.9],'gamma':[0.9],'epsilon':[.9]}

      decay_over (boolean) -> to apply the decay technique or not
      decay_factor (float) -> due to the decay equation we need the decay_factor, the Equation (parameter*(1-parameter*decay_factor) )

  Output:
      best_params (dict) -> with the best paramters
      best_AVGtime (float) -> the best avarage time
      best_AVGpenalties (float) -> the least penalty value
      best_frame (list)

  """
  best_AVGtime , best_AVGpenalties= 999999,999999
  best_frame =None
  best_params={}

  for alpha in parameters['alpha']:
        for gamma in parameters['gamma']:
            for epsilon in parameters['epsilon']:
              frames, AVG_timesteps, AVG_penalities = train_model(env_name, alpha_para = alpha, gamma_para = gamma, epsilon_para = epsilon,decay_over=decay_over,decay_factor=decay_factor)
              if AVG_penalities <= best_AVGpenalties:
                    if AVG_timesteps <= best_AVGtime :
                      best_AVGtime ,best_AVGpenalties = AVG_timesteps, AVG_penalities
                      best_params = {'alpha':alpha,'gamma':gamma,'epsilon':epsilon}
                      best_frame = frames

  return best_params, best_AVGtime ,best_AVGpenalties, best_frame

In [38]:
env_name = "FrozenLake-v1"
params = {'alpha':[0.9,0.6,0.3],'gamma':[0.9,0.6,0.3],'epsilon':[0.9,0.6,0.3]} #[0.9,0.6,0.3]
best_params, best_AVGtime ,best_AVGpenalties, best_frame = grid_search(env_name=env_name,parameters=params,decay_over=False,decay_factor=.1)


Episode: 100000


({'alpha': 0.9, 'epsilon': 0.9, 'gamma': 0.9},
 5.1,
 0.0,
 {'action': 2,
  'frame': '  (Right)\nS\x1bF\x1bFF\nFHFH\nFFFH\nHFFG\n',
  'reward': 0.0,
  'state': 1})

In [40]:
print('Best_parameters:', best_params)
print('Average timesteps per episode:', best_AVGtime)
print('Average penalties per episode:', best_AVGpenalties)

Best_parameters: {'alpha': 0.9, 'gamma': 0.9, 'epsilon': 0.9}
Average timesteps per episode: 5.1
Average penalties per episode: 0.0


In [39]:
print_frames(best_frame)

Streaming output truncated to the last 5000 lines.
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 11
State: 1
Action: 2
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 12
State: 2
Action: 2
Reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG

Timestep: 13
State: 1
Action: 0
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 14
State: 5
Action: 2
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 15
State: 4
Action: 2
Reward: 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG

Timestep: 16
State: 5
Action: 1
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 17
State: 0
Action: 2
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 18
State: 0
Action: 2
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 19
State: 0
Action: 2
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 20
State: 1
Action: 2
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 21
State: 2
Action: 2
Reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG

Timestep: 22
State: 1
Action: 0
Reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

Timestep: 23
State: 1

In [41]:
env_name = "Taxi-v3"
params = {'alpha':[0.9,0.6,0.3],'gamma':[0.9,0.6,0.3],'epsilon':[0.9,0.6,0.3]} #[0.9,0.6,0.3]
best_params, best_AVGtime ,best_AVGpenalties, best_frame = grid_search(env_name=env_name,parameters=params,decay_over=False,decay_factor=.1)

Episode: 100000


In [42]:
print('Best_parameters:', best_params)
print('Average timesteps per episode:', best_AVGtime)
print('Average penalties per episode:', best_AVGpenalties)

Best_parameters: {'alpha': 0.6, 'gamma': 0.3, 'epsilon': 0.9}
Average timesteps per episode: 12.43
Average penalties per episode: 0.0


In [43]:
print_frames(best_frame)

Streaming output truncated to the last 5000 lines.
|Y| : |B: |
+---------+
  (North)

Timestep: 859
State: 217
Action: 1
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| :_: : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

Timestep: 860
State: 237
Action: 2
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : :_: : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

Timestep: 861
State: 257
Action: 2
Reward: -1
+---------+
|R: | : :G|
| : |_: : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

Timestep: 862
State: 157
Action: 1
Reward: -1
+---------+
|R: |_: :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

Timestep: 863
State: 57
Action: 1
Reward: -1
+---------+
|R: | :_:G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

Timestep: 864
State: 77
Action: 2
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

Timestep: 865
State: 97
Action: 2
Reward: -1
+---------+
|R: | : :G|
| : |